<p align="center">
<img src="https://mistral.ai/images/logo_hubc88c4ece131b91c7cb753f40e9e1cc5_2589_256x0_resize_q97_h2_lanczos_3.webp" width="100"/>
<img src="https://cdn.iconscout.com/icon/free/png-256/free-python-3521655-2945099.png?f=webp&w=256" width="35"/>
</p>

<h1 align="center">Génération d'un Dataset  pour proceder au fine-tuning d'un modèle Mistral.ai</h1>

---
<p style='text-align: justify;'>
Ce notebook est un outil exploratoire de génération d'un dataset pour proceder au fine-tuning d'un modèle Mistral.ai. Il permet de générer un grand nombre de dialogue sur base d'un prompt. Ce prompt est construit sur base d'une question et d'une réponse. Le prompt est ensuite utilisé pour générer un grand nombre de dialogue. Ces dialogues sont ensuite stockés dans un fichier json. Ce fichier json est ensuite utilisé pour proceder au fine-tuning d'un modèle Mistral.ai.

Dans cet exemple, nous allons générer un dataset pour un CV. La question est "Comment je m'appelle" et la réponse est "Tu t'appelles [nom] [prénom]". Le prompt est ensuite utilisé pour générer 50 variations de la question et de la réponse. Le but est de montrer la capacité du modèle à générer un dataset de qualité de dialogue sur base d'un prompt.
</p>

---

## Installation et importation des données du CV

Ce morceaux de code permet de charger et de pouvoir naviguer dans les données disponibles dans le json

In [1]:
import json

# Chemin vers le fichier JSON
chemin_fichier = 'data.json'

# Ouvrir et lire le fichier JSON
with open(chemin_fichier, 'r') as fichier:
    CV_data = json.load(fichier)

question = "Comment je m'appelle"
reponse = f"Tu t'appelles {CV_data[0]["Identite"]["Prenom"] + " " + CV_data[0]["Identite"]["Nom"]}"

CV_data[0]["Identite"]["Prenom"] + " " + CV_data[0]["Identite"]['Nom']

'Jean-Baptiste ASTRUZ'

## Création d'un prompt permettant de demander la génération d'un nombre arbitrairement grand de dialogue

Ce prompt à pour but de générer un ordre dynamique qui puisse permettre de générer un grand nombre de réponse sur base d'un grand nombre de questions

In [2]:
prompt = f"""je souhaite que tu me génères exactement 50 variations de la question {question}?. De plus, je souhaite que pour chaque variations, tu puisses donner la paraphrase de la réponse qui est "{reponse}.". N'hesites pas à reformuler la réponse""" + """
Génère la réponse sous forme de JSON uniquement avec la structure suivante:
{
"messages": [
{
"role": "user",
"content" : "Variation1 de la question"
},
{
"role" : "assistant",
"content" : "Variation1 de la réponse"
},
{
"role": "user",
"content" : "Variation2 de la question"
},
{
"role" : "assistant",
"content" : "Variation2 de la réponse"
},
{
"role": "user",
"content" : "Variation3 de la question"
},
{
"role" : "assistant",
"content" : "Variation3 de la réponse"
},
...
}
Ne rajoute pas de texte en dehors de la structure JSON. ni avant ni après. Ne met pas de mise en forme markdown. De plus, tu génères EXACTEMENT 50 variations. Tu ne parles qu'en français.
"""

## Installation et importation des packages nécessaires

On récupère les bons packages pour pouvoir faire le travail d'IA

In [3]:
from mistralai.client import MistralClient # Import the MistralClient from the mistralai.client module
from mistralai.models.chat_completion import ChatMessage # Import the ChatMessage from the mistralai.models.chat_completion module
from dotenv import load_dotenv # Import the load_dotenv function from the dotenv module
import os # Import the os module

## Définition du client Mistral.ai et du modèle à utiliser
On défini le client et le modèle d'IA que l'on va utiliser

In [4]:
load_dotenv()
model = "open-codestral-mamba"
api_key= os.getenv('MISTRAL_API_KEY')
client = MistralClient(api_key=api_key, timeout=180)
responses = []

On execute le prompt pour pouvoir récupérer les exemples de diaolgue fournis en JSON

In [5]:
messages = [
    ChatMessage(role="user", content=prompt)
]
responses.append(client.chat(model=model, messages=messages, response_format={"type": "json_object"}))

## Enregistrement des résultats
On va simplement enregistrer les résultats pour pouvoir les charger plus tard

In [6]:
json_text = responses[0].choices[0].message.content

json_object = json.loads(json_text)

# Enregistrer les données JSON dans un fichier
with open(f'{question.replace(" ", "_")}.json', 'w') as json_file:
    json.dump(json_object, json_file, indent=4)

print(f"Le fichier JSON a été sauvegardé avec succès dans le fichier {question.replace(" ", "_")}.json.")

Le fichier JSON a été sauvegardé avec succès dans le fichier Comment_je_m'appelle.json.


## Vérification du nombre de messages générés

In [7]:
import json

# Charger les données JSON depuis le fichier
with open(f'{question.replace(" ", "_")}.json', 'r') as json_file:
    data = json.load(json_file)

# Initialiser un dictionnaire pour stocker le nombre d'occurrences de chaque rôle
role_count = {}

# Parcourir la liste de messages
for message in data['messages']:
    role = message['role']
    # Si le rôle est déjà dans le dictionnaire, incrémenter le compteur
    if role in role_count:
        role_count[role] += 1
    # Sinon, ajouter le rôle au dictionnaire avec un compteur initial de 1
    else:
        role_count[role] = 1

# Afficher le nombre d'occurrences de chaque rôle
for role, count in role_count.items():
    print(f"Le rôle '{role}' apparaît {count} fois.")

Le rôle 'user' apparaît 42 fois.
Le rôle 'assistant' apparaît 42 fois.
